<a href="https://colab.research.google.com/github/dandamudi-geeta/Quantum-Computing/blob/main/2348512_QC(Lab7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 10.0 MB/s eta 0:00:00


In [14]:

!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 93.4 MB/s eta 0:00:00


In [15]:
from qiskit_aer import AerSimulator

In [35]:
from qiskit import QuantumCircuit,transpile
from qiskit.visualization import plot_histogram


In [36]:
# Step 1: Define the Deutsch function (Oracle)
def deutsch_function(case: str) -> QuantumCircuit:
    """
    Returns a quantum circuit implementing the oracle for the Deutsch problem.
    case: 'constant_0', 'constant_1', 'balanced_0', 'balanced_1'
    """
    oracle = QuantumCircuit(2)
    if case == "constant_0":  # f(x) = 0
        pass  # No operation needed
    elif case == "constant_1":  # f(x) = 1
        oracle.x(1)  # Apply X gate to the second qubit
    elif case == "balanced_0":  # f(x) = x
        oracle.cx(0, 1)  # Apply CNOT with qubit 0 as control and qubit 1 as target
    elif case == "balanced_1":  # f(x) = 1 ⊕ x
        oracle.cx(0, 1)
        oracle.x(1)  # Apply X gate after the CNOT
    else:
        raise ValueError("Invalid case. Choose from 'constant_0', 'constant_1', 'balanced_0', 'balanced_1'.")
    return oracle

In [37]:
# Step 2: Define Deutsch's algorithm circuit
def deutsch_algorithm(case: str) -> QuantumCircuit:
    """
    Creates a quantum circuit for Deutsch's algorithm.
    case: 'constant_0', 'constant_1', 'balanced_0', 'balanced_1'
    """
    # Initialize a quantum circuit with 2 qubits and 1 classical bit
    qc = QuantumCircuit(2, 1)

    # Step 2.1: Prepare the input state
    qc.h(0)  # Apply Hadamard to the first qubit
    qc.x(1)  # Flip the second qubit to |1⟩
    qc.h(1)  # Apply Hadamard to the second qubit

    # Step 2.2: Add the oracle
    oracle = deutsch_function(case)
    qc.compose(oracle, inplace=True)

    # Step 2.3: Apply Hadamard to the first qubit
    qc.h(0)

    # Step 2.4: Measure the first qubit
    qc.measure(0, 0)

    return qc

In [38]:
# Step 3: Function to interpret the results
def interpret_result(circuit: QuantumCircuit) -> str:
    """
    Runs the Deutsch algorithm circuit once and interprets the result.
    Returns "constant" or "balanced".
    """
    simulator = AerSimulator()
    compiled_circuit = transpile(circuit, simulator)
    result = simulator.run(compiled_circuit, shots=1).result()
    counts = result.get_counts()

    # Determine result based on measurement
    output = list(counts.keys())[0]  # Get the measurement result
    return "constant" if output == "0" else "balanced"

In [39]:
# Example Usage
case = "balanced_0"  # Change to test other cases
circuit = deutsch_algorithm(case)

In [40]:
# Display the circuit
print("Deutsch Algorithm Circuit:")
print(circuit.draw(output='text'))



Deutsch Algorithm Circuit:
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐└───┘└╥┘
q_1: ┤ X ├┤ H ├┤ X ├──────╫─
     └───┘└───┘└───┘      ║ 
c: 1/═════════════════════╩═
                          0 


In [41]:
# Interpret the result
result = interpret_result(circuit)
print(f"The function is {result}.")

The function is balanced.
